In [1]:
import pandas as pd
import pickle as pk
import re
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid", {"axes.facecolor": ".9"})

df = pd.read_csv(f"./../../outputs/cities_prediction.csv")
common_columns = ["Geoname ID", "Name", "Population", "Elevation", "Coordinates", "Latitude", "Longitude", "city"]
expe_columns = [col for col in df.keys() if "lat_predicted" in col or "lon_predicted" in col]
selected_columns = common_columns + expe_columns
selected_columns = [col for col in selected_columns if col in df.columns]

df = df[common_columns + expe_columns]
df.head()


/tmp/ipykernel_17816/1368085679.py:8: DtypeWarning: Columns (10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"./../../outputs/cities_prediction.csv")


,Geoname ID,Name,Population,Elevation,Coordinates,Latitude,Longitude,city,mistralai/Mistral-7B-v0.1_float16_gps_fr_lat_predicted,mistralai/Mistral-7B-v0.1_float16_gps_fr_lon_predicted,...,Qwen/Qwen2.5-32B-Instruct_int4_gps_en_lat_predicted,Qwen/Qwen2.5-32B-Instruct_int4_gps_en_lon_predicted,Qwen/Qwen2.5-72B_int4_gps_fr_lat_predicted,Qwen/Qwen2.5-72B_int4_gps_fr_lon_predicted,Qwen/Qwen2.5-72B_int4_gps_en_lat_predicted,Qwen/Qwen2.5-72B_int4_gps_en_lon_predicted,Qwen/Qwen2.5-72B-Instruct_int4_gps_fr_lat_predicted,Qwen/Qwen2.5-72B-Instruct_int4_gps_fr_lon_predicted,Qwen/Qwen2.5-72B-Instruct_int4_gps_en_lat_predicted,Qwen/Qwen2.5-72B-Instruct_int4_gps_en_lon_predicted
0,3037943,Ambérieu-en-Bugey,13260,NaN,"45.95932, 5.3516",45.95932,5.35160,Ambérieu-en-Bugey,45.916389,5.299722,...,46.2500,4.7167,46.0833,5.3667,46.1667,5.2500,NaN,NaN,NaN,NaN
1,3031009,Bourg-en-Bresse,45340,NaN,"46.20574, 5.2258",46.20574,5.22580,Bourg-en-Bresse,46.166667,5.033333,...,46.2333,4.7167,46.2000,5.1833,46.2000,5.1833,NaN,NaN,NaN,NaN
2,3033685,Bellegarde-sur-Valserine,11699,NaN,"46.10787, 5.82421",46.10787,5.82421,Bellegarde-sur-Valserine,45.933056,5.899722,...,46.1500,5.3833,46.1667,5.9667,46.1167,6.0333,NaN,NaN,NaN,NaN
3,2988888,Oyonnax,25697,NaN,"46.25917, 5.65727",46.25917,5.65727,Oyonnax,46.466667,5.866667,...,46.7167,3.9500,NaN,NaN,46.1667,5.6667,NaN,NaN,NaN,NaN
4,3026194,Château-Thierry,15938,NaN,"49.04636, 3.40304",49.04636,3.40304,Château-Thierry,NaN,NaN,...,49.2833,3.3667,49.2500,3.2500,49.2500,3.1800,NaN,NaN,NaN,NaN


In [2]:
import geopandas as gpd

gdf_france = gpd.read_file("https://github.com/gregoiredavid/france-geojson/raw/refs/heads/master/metropole.geojson")

In [3]:
from shapely.geometry import Point
import pandas as pd

df["geometry"] = df.apply(lambda row: Point(row["Longitude"], row["Latitude"]), axis=1)

lat_columns = [col for col in df.columns if "lat_predicted" in col]
lon_columns = [col for col in df.columns if "lon_predicted" in col]
lat_lon_pairs = [(lat, lat.replace("lat_predicted", "lon_predicted")) for lat in lat_columns if lat.replace("lat_predicted", "lon_predicted") in lon_columns]
for lat_col, lon_col in lat_lon_pairs:
    geom_col = f"geom_{lat_col.replace('lat_predicted', '')}"  # Naming convention
    df[geom_col] = df.apply(lambda row: Point(row[lon_col], row[lat_col]), axis=1)

gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")


# Détecter les villes en bordure en vérifiant si elles touchent la frontière
# gdf_bordure = gdf[gdf.touches(gdf_france)]
# gdf_bordure
# gdf_bordure.to_file("./../../outputs/villes_bordure.shp")

/tmp/ipykernel_17816/1685585843.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[geom_col] = df.apply(lambda row: Point(row[lon_col], row[lat_col]), axis=1)
/tmp/ipykernel_17816/1685585843.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[geom_col] = df.apply(lambda row: Point(row[lon_col], row[lat_col]), axis=1)
/tmp/ipykernel_17816/1685585843.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all

In [4]:
experiments = [col.split("_lat_predicted")[0] for col in gdf.columns if "lat_predicted" in col]
experiments = list(set(experiments))
experiments

['meta-llama/Llama-2-7b-hf_float16_gps_en',
 'meta-llama/Llama-2-13b-hf_int8_gps_en',
 'Qwen/Qwen2.5-0.5B_int8_gps_fr',
 'Qwen/Qwen2.5-14B-Instruct_int4_gps_fr',
 'meta-llama/Llama-3.2-1B_int4_gps_en',
 'meta-llama/Llama-3.2-3B_int4_gps_en',
 'meta-llama/Llama-3.1-8B-Instruct_int8_gps_fr',
 'mistralai/Mistral-7B-Instruct-v0.1_float16_gps_fr',
 'meta-llama/Llama-2-13b-chat-hf_int8_gps_en',
 'mistralai/Mistral-7B-Instruct-v0.2_int4_gps_fr',
 'meta-llama/Llama-2-7b-chat-hf_int8_gps_en',
 'Qwen/Qwen2.5-0.5B-Instruct_int4_gps_en',
 'Qwen/Qwen2.5-32B_int4_gps_fr',
 'meta-llama/Llama-3.1-8B_int4_gps_en',
 'meta-llama/Llama-3.1-8B_int4_gps_fr',
 'Qwen/Qwen2.5-0.5B_int4_gps_fr',
 'meta-llama/Llama-3.2-3B-Instruct_int8_gps_en',
 'mistralai/Mistral-7B-Instruct-v0.1_float16_gps_en',
 'mistralai/Mistral-Small-24B-Instruct-2501_int8_gps_fr',
 'meta-llama/Llama-3.1-70B-Instruct_int4_gps_fr',
 'meta-llama/Llama-3.1-8B-Instruct_float16_gps_en',
 'Qwen/Qwen2.5-0.5B_int4_gps_en',
 'mistralai/Mistral-7B-v

In [5]:
import folium

# Discretisation de l'espace : 
# expe = "mistralai/Mistral-7B-Instruct-v0.3_float16_gps_fr"

# 2nd best models (instruct)
# expe = "mistralai/Mistral-Small-24B-Instruct-2501_float16_gps_fr"

# expe = "mistralai/Mistral-Small-24B-Base-2501_float16_gps_fr"

# expe = "mistralai/Mistral-Nemo-Base-2407_float16_gps_fr"

expe = "meta-llama/Llama-2-7b-chat-hf_int4_gps_en"

for expe in experiments:

    df_clean = gdf.dropna(subset=[
        f"{expe}_lat_predicted",
        f"{expe}_lon_predicted"
    ])


    m = folium.Map(location=[46.6031, 1.8883], zoom_start=6)
    for _, row in df_clean.iterrows():
        # Ajouter le point réel
        folium.CircleMarker(location=[row["Latitude"], row["Longitude"]],
                            color="blue", radius=4, fill=True, fill_color="blue", popup=row["city"]).add_to(m)
        
        # Ajouter le point prédit
        folium.CircleMarker(location=[row[f"{expe}_lat_predicted"], row[f"{expe}_lon_predicted"]],
                            color="red", radius=4, fill=True, fill_color="red", popup=row["city"]).add_to(m)

        # Ajouter une ligne entre les deux
        folium.PolyLine([(row["Latitude"], row["Longitude"]), 
                        (row[f"{expe}_lat_predicted"], row[f"{expe}_lon_predicted"])], 
                        color="green", weight=2, opacity=0.7).add_to(m)

    # m
    m.save(f"./../../docs/shifting_maps_{expe.split('/')[-1]}.html")

### generate an index.html file

In [6]:
from pathlib import Path
import sys
import os

root_path = Path.cwd().resolve().parents[1]  # adjust if needed
sys.path.append(str(root_path))

from pipeline_config import list_of_models, list_of_prompts, quantizations, mistral_models


# Create the docs folder if it doesn't exist
docs_path = Path("./../../docs")

# Start building HTML
html = """
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Experimentation Table</title>
    <style>
        table { border-collapse: collapse; width: 100%; }
        th, td { border: 1px solid #ccc; padding: 8px; text-align: left; }
        th { background-color: #f2f2f2; }
        a { text-decoration: none; color: blue; }
    </style>
</head>
<body>
    <h2>Model Experimentations</h2>
    <table>
        <tr>
            <th>Model Family</th>
            <th>Model Size</th>
            <th>Quantization</th>
            <th>Lang Prompt</th>
            <th>Fine-tuning</th>
            <th>Map Link</th>
        </tr>
"""

# Generate rows
for model in list_of_models:
    model_family = model.split("/")[0] if "/" in model else model.split("-")[0]
    model_name = model.split("/")[-1]
    model_size = model_name.split("-")[0].replace("Qwen2.5", "") if "Qwen" in model else model_name.split("-")[1]
    if "instruct" in model.lower():
        finetuned = "Instruct"
    elif "chat" in model.lower():
        finetuned = "Chat"
    else:
        finetuned = '-'
    
    for quant in quantizations:
        for lang_prompt in list_of_prompts:
            map_filename = f"shifting_maps_{model_name}_{quant}_{lang_prompt}.html"
            map_path = docs_path / map_filename
            link = f'<a href="{map_filename}">{map_filename}</a>' if map_path.exists() else "-"
            
            html += f"""
            <tr>
                <td>{model_family}</td>
                <td>{model_size}</td>
                <td>{quant}</td>
                <td>{lang_prompt}</td>
                <td>{finetuned}</td>
                <td>{link}</td>
            </tr>
            """

html += """
    </table>
</body>
</html>
"""

# Write to index.html
with open(docs_path / "index.html", "w", encoding="utf-8") as f:
    f.write(html)

print("✅ index.html has been created in the docs/ folder.")



✅ index.html has been created in the docs/ folder.


## Analyze Lon shiffted because of minus (-1)

In [7]:
lon_columns = [col for col in df.columns if col.endswith("_lon_predicted")]
lon_columns.append("Longitude")
negative_count = (df[lon_columns] < 0).sum().sort_values(ascending=False)
print(f"Number of real negative Lon: {negative_count['Longitude']}")
print("\n ----------------")
negative_count

Number of real negative Lon: 132

 ----------------


meta-llama/Llama-2-7b-chat-hf_int4_gps_en_lon_predicted               251
mistralai/Mistral-Small-24B-Base-2501_float16_gps_en_lon_predicted    157
mistralai/Mistral-Small-24B-Base-2501_int8_gps_en_lon_predicted       154
mistralai/Mistral-7B-v0.3_float16_gps_en_lon_predicted                146
mistralai/Mistral-7B-v0.3_int8_gps_en_lon_predicted                   141
                                                                     ... 
meta-llama/Llama-2-13b-hf_int8_gps_fr_lon_predicted                     0
meta-llama/Llama-2-13b-hf_float16_gps_fr_lon_predicted                  0
meta-llama/Llama-2-7b-hf_int4_gps_en_lon_predicted                      0
meta-llama/Llama-2-7b-hf_int8_gps_fr_lon_predicted                      0
mistralai/Mistral-7B-v0.1_float16_gps_fr_lon_predicted                  0
Length: 159, dtype: int64

In [8]:
expe = "mistralai/Mistral-7B-v0.1_float16_gps_fr"
expe = "mistralai/Mistral-Small-24B-Base-2501_float16_gps_fr"

df[df["Longitude"] <0][["Coordinates", f"{expe}_output", f"{expe}_lat_predicted", f"{expe}_lon_predicted"]].to_csv(f"./outputs/TMP/analyse_shift_lat_{expe.split('/')[-1]}.csv")

KeyError: "['mistralai/Mistral-Small-24B-Base-2501_float16_gps_fr_output'] not in index"